In [1]:
import dolfinx as dlx
import numpy as np
import pyvista as pv
import scifem as dlx_helper
import ufl
from mpi4py import MPI

In [2]:
square_mesh = dlx.mesh.create_unit_square(MPI.COMM_WORLD, 8, 8, dlx.mesh.CellType.triangle)
vertices = square_mesh.geometry.x
simplices = square_mesh.geometry.dofmap

is_left_boundary = np.where(vertices[:, 0] == 0)[0]
is_right_boundary = np.where(vertices[:, 0] == 1)[0]
is_bottom_boundary = np.where(vertices[:, 1] == 0)[0]
is_top_boundary = np.where(vertices[:, 1] == 1)[0]

alpha_left_boundary_values = np.zeros(is_left_boundary.size)
alpha_right_boundary_values = np.ones(is_right_boundary.size)
alpha_bottom_boundary_values = vertices[is_bottom_boundary, 0]
alpha_top_boundary_values = vertices[is_top_boundary, 0]

beta_left_boundary_values = vertices[is_left_boundary, 1]
beta_right_boundary_values = vertices[is_right_boundary, 1]
beta_bottom_boundary_values = np.zeros(is_bottom_boundary.size)
beta_top_boundary_values = np.ones(is_top_boundary.size)

In [3]:
alpha_left_boundary = pv.PolyData(vertices[is_left_boundary])
alpha_left_boundary.point_data["values"] = alpha_left_boundary_values
alpha_right_boundary = pv.PolyData(vertices[is_right_boundary])
alpha_right_boundary.point_data["values"] = alpha_right_boundary_values
alpha_bottom_boundary = pv.PolyData(vertices[is_bottom_boundary])
alpha_bottom_boundary.point_data["values"] = alpha_bottom_boundary_values
alpha_top_boundary = pv.PolyData(vertices[is_top_boundary])
alpha_top_boundary.point_data["values"] = alpha_top_boundary_values

beta_left_boundary = pv.PolyData(vertices[is_left_boundary])
beta_left_boundary.point_data["values"] = beta_left_boundary_values
beta_right_boundary = pv.PolyData(vertices[is_right_boundary])
beta_right_boundary.point_data["values"] = beta_right_boundary_values
beta_bottom_boundary = pv.PolyData(vertices[is_bottom_boundary])
beta_bottom_boundary.point_data["values"] = beta_bottom_boundary_values
beta_top_boundary = pv.PolyData(vertices[is_top_boundary])
beta_top_boundary.point_data["values"] = beta_top_boundary_values

In [4]:
pv_mesh = pv.PolyData.from_regular_faces(vertices, simplices)
plotter = pv.Plotter(window_size=[600, 600])
plotter.add_mesh(pv_mesh, style="wireframe", color="gray")
plotter.add_points(alpha_left_boundary, point_size=20, render_points_as_spheres=True)
plotter.add_points(alpha_right_boundary, point_size=20, render_points_as_spheres=True)
plotter.add_points(alpha_bottom_boundary, point_size=20, render_points_as_spheres=True)
plotter.add_points(alpha_top_boundary, point_size=20, render_points_as_spheres=True)
plotter.view_xy()
plotter.show()

Widget(value='<iframe src="http://localhost:44327/index.html?ui=P_0x7f9e58e2b620_0&reconnect=auto" class="pyvi…

In [5]:
pv_mesh = pv.PolyData.from_regular_faces(vertices, simplices)
plotter = pv.Plotter(window_size=[600, 600])
plotter.add_mesh(pv_mesh, style="wireframe", color="gray")
plotter.add_points(beta_left_boundary, point_size=20, render_points_as_spheres=True)
plotter.add_points(beta_right_boundary, point_size=20, render_points_as_spheres=True)
plotter.add_points(beta_bottom_boundary, point_size=20, render_points_as_spheres=True)
plotter.add_points(beta_top_boundary, point_size=20, render_points_as_spheres=True)
plotter.view_xy()
plotter.show()

Widget(value='<iframe src="http://localhost:44327/index.html?ui=P_0x7f9e44cd0050_1&reconnect=auto" class="pyvi…

In [6]:
function_space = dlx.fem.functionspace(square_mesh, ("Lagrange", 1))
vertex_to_dof_map = dlx_helper.vertex_to_dofmap(function_space)
dof_to_vertex_map = dlx_helper.dof_to_vertexmap(function_space)

trial_function = ufl.TrialFunction(function_space)
test_function = ufl.TestFunction(function_space)
weak_form = ufl.inner(ufl.grad(trial_function), ufl.grad(test_function)) * ufl.dx
rhs_constant = dlx.fem.Constant(square_mesh, 0.0)
right_hand_side = rhs_constant * test_function * ufl.dx

In [7]:
left_boundary_dofs = vertex_to_dof_map[is_left_boundary]
right_boundary_dofs = vertex_to_dof_map[is_right_boundary]
bottom_boundary_dofs = vertex_to_dof_map[is_bottom_boundary]
top_boundary_dofs = vertex_to_dof_map[is_top_boundary]

bc_function = dlx.fem.Function(function_space)
bc_function.x.array[:] = 0
bc_function.x.array[left_boundary_dofs] = alpha_left_boundary_values
bc_function.x.array[right_boundary_dofs] = alpha_right_boundary_values
bc_function.x.array[bottom_boundary_dofs] = alpha_bottom_boundary_values
bc_function.x.array[top_boundary_dofs] = alpha_top_boundary_values

boundary_alpha_left = dlx.fem.dirichletbc(bc_function, left_boundary_dofs)
boundary_alpha_right = dlx.fem.dirichletbc(bc_function, right_boundary_dofs)
boundary_alpha_bottom = dlx.fem.dirichletbc(bc_function, bottom_boundary_dofs)
boundary_alpha_top = dlx.fem.dirichletbc(bc_function, top_boundary_dofs)
boundary_conditions_alpha = [
    boundary_alpha_left,
    boundary_alpha_right,
    boundary_alpha_bottom,
    boundary_alpha_top,
]

In [8]:
problem = dlx.fem.petsc.LinearProblem(
    weak_form,
    right_hand_side,
    boundary_conditions_alpha,
    petsc_options={"ksp_type": "cg", "pc_type": "ilu"},
)
solution = problem.solve()

In [9]:
solution_on_vertices = solution.x.array[dof_to_vertex_map]
alpha_solution = pv.PolyData(vertices)
alpha_solution.point_data["values"] = solution_on_vertices

pv_mesh = pv.PolyData.from_regular_faces(vertices, simplices)
plotter = pv.Plotter(window_size=[600, 600])
plotter.add_mesh(pv_mesh, style="wireframe", color="gray")
plotter.add_points(alpha_solution, point_size=10, render_points_as_spheres=True)
plotter.view_xy()
plotter.show()

Widget(value='<iframe src="http://localhost:44327/index.html?ui=P_0x7f9e44cd2350_2&reconnect=auto" class="pyvi…

In [10]:
bc_function = dlx.fem.Function(function_space)
bc_function.x.array[:] = 0
bc_function.x.array[left_boundary_dofs] = beta_left_boundary_values
bc_function.x.array[right_boundary_dofs] = beta_right_boundary_values
bc_function.x.array[bottom_boundary_dofs] = beta_bottom_boundary_values
bc_function.x.array[top_boundary_dofs] = beta_top_boundary_values

boundary_beta_left = dlx.fem.dirichletbc(bc_function, left_boundary_dofs)
boundary_beta_right = dlx.fem.dirichletbc(bc_function, right_boundary_dofs)
boundary_beta_bottom = dlx.fem.dirichletbc(bc_function, bottom_boundary_dofs)
boundary_beta_top = dlx.fem.dirichletbc(bc_function, top_boundary_dofs)
boundary_conditions_beta = [
    boundary_beta_left,
    boundary_beta_right,
    boundary_beta_bottom,
    boundary_beta_top,
]

In [11]:
problem = dlx.fem.petsc.LinearProblem(
    weak_form,
    right_hand_side,
    boundary_conditions_beta,
    petsc_options={"ksp_type": "cg", "pc_type": "ilu"},
)
solution = problem.solve()

In [12]:
solution_on_vertices = solution.x.array[dof_to_vertex_map]
alpha_solution = pv.PolyData(vertices)
alpha_solution.point_data["values"] = solution_on_vertices

pv_mesh = pv.PolyData.from_regular_faces(vertices, simplices)
plotter = pv.Plotter(window_size=[600, 600])
plotter.add_mesh(pv_mesh, style="wireframe", color="gray")
plotter.add_points(alpha_solution, point_size=10, render_points_as_spheres=True)
plotter.view_xy()
plotter.show()

Widget(value='<iframe src="http://localhost:44327/index.html?ui=P_0x7f9e44cd3390_3&reconnect=auto" class="pyvi…